In [462]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from mlxtend.feature_selection import ColumnSelector
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [463]:
# Displaying all columns
pd.set_option('display.max_columns', None)

#Display all data in columns
# pd.set_option("display.max_colwidth", None)

# np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.expand_frame_repr", True)
# pd.set_option('display.width', 1000)


In [464]:
#Storing Accuracy

data_accuracy = pd.DataFrame(columns=['Algo', 'Accuracy(KFold)', 'Test', 'Train', 'Recall', 'Precision', 'F1', 'ROC_AUC_SCORE'])
def store(data):
    data_accuracy.loc[len(data_accuracy.index)] = data
    data_accuracy.to_csv('loan_prediction_metric_scores.csv')
    print(data_accuracy)



def get_info_pipeline(pipe_name):
    sub_pipe = pipe_name[:-1]
    transformed_df = pd.DataFrame(
        data=sub_pipe.fit_transform(X_train, y_train),
        columns=sub_pipe.get_feature_names_out(),
    )
    print(transformed_df)

In [465]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [466]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [467]:
df1.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [468]:
#Renaming the 'movie_id' col in df1 to 'id' of df2 

In [469]:
df1.rename(columns = {'movie_id':'id'}, inplace = True)

In [470]:
df1.head(1)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [471]:
#joining the credits amd movie dataframes

In [472]:
df2 = pd.merge(df1, df2, on='id')

In [473]:
df2.head(1)

,id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [474]:
#Drpping columns

In [475]:
df2.columns

Index(['id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count'],
      dtype='object')

In [476]:
df2.drop(columns=['status', 'runtime'], axis=1, inplace=True)

In [477]:
df2.drop(columns=['title_x'], axis=1, inplace=True)

In [478]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,title_y,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800


In [479]:
df2.drop(columns=['title_y'], axis=1, inplace=True)

In [480]:
df2.shape

(4803, 19)

In [481]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800


In [482]:
df2.drop(columns=['revenue', 'release_date'], axis=1, inplace=True)

In [483]:
df2.drop(columns=['budget', 'homepage', 'popularity', 'spoken_languages', 'production_countries', 'vote_average', 'vote_count'], axis=1, inplace=True)

In [484]:
df2.shape

(4803, 10)

In [485]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,production_companies,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.


In [486]:
df2.isnull().sum()

id                        0
cast                      0
crew                      0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  3
production_companies      0
tagline                 844
dtype: int64

In [487]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4803 non-null   int64 
 1   cast                  4803 non-null   object
 2   crew                  4803 non-null   object
 3   genres                4803 non-null   object
 4   keywords              4803 non-null   object
 5   original_language     4803 non-null   object
 6   original_title        4803 non-null   object
 7   overview              4800 non-null   object
 8   production_companies  4803 non-null   object
 9   tagline               3959 non-null   object
dtypes: int64(1), object(9)
memory usage: 412.8+ KB


In [488]:
df2.drop(columns='production_companies', axis=1, inplace=True)

In [489]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.


In [490]:
df2.shape

(4803, 9)

In [491]:
#Dropping null rows

In [492]:
df2.isnull().sum()

id                     0
cast                   0
crew                   0
genres                 0
keywords               0
original_language      0
original_title         0
overview               3
tagline              844
dtype: int64

In [493]:
df2.drop(columns='tagline', axis=1, inplace=True)

In [494]:
df2.dropna(inplace=True)

In [495]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [496]:
df2.isnull().sum()

id                   0
cast                 0
crew                 0
genres               0
keywords             0
original_language    0
original_title       0
overview             0
dtype: int64

In [497]:
#Preprocessing

In [498]:
##Genre

In [499]:
df2.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [500]:
import ast

In [501]:
def fetch_genres(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
    return ls

In [502]:
df2['genres'] = df2['genres'].apply(fetch_genres)

In [503]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [504]:
df2.iloc[3, 3]

['Action', 'Crime', 'Drama', 'Thriller']

In [505]:
##Crew

In [506]:
def fetch_dir_and_prod(string):
    ls = []
    for i in  ast.literal_eval(string):
        if i['job']=='Director':
            ls.append(i['name'])
            break
    return ls

In [507]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [508]:
msg = " "
msg = df2.loc[4].crew

In [509]:
msg

'[{"credit_id": "52fe479ac3a36847f813eaa3", "department": "Writing", "gender": 2, "id": 7, "job": "Screenplay", "name": "Andrew Stanton"}, {"credit_id": "52fe479ac3a36847f813ea65", "department": "Directing", "gender": 2, "id": 7, "job": "Director", "name": "Andrew Stanton"}, {"credit_id": "58c69e819251411d30017673", "department": "Crew", "gender": 2, "id": 7879, "job": "Thanks", "name": "John Lasseter"}, {"credit_id": "52fe479ac3a36847f813eac5", "department": "Production", "gender": 2, "id": 490, "job": "Producer", "name": "Colin Wilson"}, {"credit_id": "58c68f2b9251411d30016cf2", "department": "Production", "gender": 2, "id": 2073, "job": "Casting", "name": "Gail Stevens"}, {"credit_id": "58c68ef19251411d62016ec3", "department": "Sound", "gender": 2, "id": 3996, "job": "Sound Re-Recording Mixer", "name": "Tom Johnson"}, {"credit_id": "52fe479ac3a36847f813ead7", "department": "Editing", "gender": 2, "id": 6958, "job": "Editor", "name": "Eric Zumbrunnen"}, {"credit_id": "58c69a9ec3a3684

In [510]:
import ast
msg = ast.literal_eval(msg)

In [511]:
for i in msg:
    if i['job']=='Director':
        print(i['name'])
        break

Andrew Stanton


In [512]:
df2['crew'] = df2['crew'].apply(fetch_dir_and_prod)

In [513]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [514]:
##keywords

In [515]:
msg = df2.iloc[0, 4]

In [516]:
msg

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [517]:
def fetch_keyowrds(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
            
    return ls

In [518]:
fetch_keyowrds(msg)

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [519]:
df2['keywords'] = df2['keywords'].apply(fetch_genres)

In [520]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [521]:
##cast (first 3)

In [522]:
msg = df2.iloc[0, 1]

In [523]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [524]:
def fetch_crew_first_three(string):
    ls = []
    cnt = 3
    for i in  ast.literal_eval(string):
        if cnt==0:
            break
        else:
            ls.append(i['name'])
        cnt=cnt-1
    return ls

In [525]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [526]:
fetch_crew_first_three(msg)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [527]:
df2['cast'] = df2['cast'].apply(fetch_crew_first_three)

In [528]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [529]:
df2.rename(columns = {'crew':'director'}, inplace = True)

In [530]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [531]:
#Preprocessing 'overview' column

#Lowercasing

df2['overview'] = df2['overview'].str.lower()


# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_html)
# print(df.head())


#Remove @

def remove_at_the_rate(text):

  ls = []
  new = []

  ls = nlp(text)

  for word in ls:
        if word.text != "@":
            new.append(word.text)

  return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_at_the_rate)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_url)
# print(df.head())


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

df2['overview'] = df2['overview'].apply(remove_punc)


#Correcting spelling

# from textblob import TextBlob

# def spell_check(text):

#     textblb = TextBlob(text)
#     return textblb.correct().string

# X['remainder__text'] = X['remainder__text'].apply(spell_check)


from autocorrect import Speller

check = Speller()

def check_spell(text):

  return check(text)

df2['overview'] = df2['overview'].apply(check_spell)


# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_stop_words)


#Removing Contradictions

import contractions

def remove_contradictions(text):

  return " ".join([contractions.fix(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(remove_contradictions)


# #Lemmetization

# from nltk.stem.wordnet import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()

def Lemmetization(text):

  return " ".join([word.lemma_ for word in nlp(text)])


# #Stemming

# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()
# def stemming(text):

#     return " ".join([ps.stem(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(Lemmetization)




In [532]:
#Tokenization of overview column

In [533]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# def token_nlp(string):
#     ls = []
#     # doc1 = nlp(string)
#     doc1 = word_tokenize(string)
#     ls.append(doc1)
#     return ls

In [534]:
# df2['overview'] = df2['overview'].apply(token_nlp)

In [535]:
df2.iloc[0, 7]

'22nd century     paraplegic marine dispatch moon pandora unique mission     become torn follow order protect alien civilization'

In [536]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [537]:
df2['tags'] = df2.sum(axis=1)

In [538]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,tags
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,19995


In [539]:
#Removing spaces between names in cast and crew column

In [540]:
def remove_spaces(ls):
    for i in range(len(ls)):
        ls[i] = ls[i].replace(' ', '')
    return ls
        # print(ls[i])

In [541]:
ls1 = df2.iloc[0, 1]

In [542]:
ls1

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [543]:
remove_spaces(ls1)

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']

In [544]:
df2.drop(columns='tags', axis=1, inplace=True)

In [545]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [546]:
df2['cast'] = df2['cast'].apply(remove_spaces)

In [547]:
df2['director'] = df2['director'].apply(remove_spaces)

In [548]:
# df2['genres'] = df2['genres'].apply(remove_spaces)

In [549]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [550]:
# df2['keywords'] = df2['keywords'].apply(remove_spaces)

In [551]:
df2['combined'] = df2['cast'] + df2['director'] + df2['genres'] + df2['keywords'] 

In [552]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."


In [553]:
msg = df2.iloc[0, 7]

In [554]:
df2.iloc[0, 8]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [555]:
def replace_big_spaces(string):
    string = string.replace('  ', '')
    return string

In [556]:
replace_big_spaces(msg)

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [557]:
df2['overview'] = df2['overview'].apply(replace_big_spaces)

In [558]:
df2.head(5)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon p...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,captain barbossa long believe dead come back l...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,Spectre,cryptic message bond ’s past send trail uncove...,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,follow death district attorney harvey dent bat...,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,John Carter,john carter war weary former military captain ...,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [559]:
msg = df2.iloc[0, 7]

In [560]:
msg

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [561]:
def list_to_string(ls):
    st=""
    
    for i in ls:
        st = ' '.join(ls)
    return st

In [562]:
msg = df2.iloc[0, 8]

In [563]:
msg

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [564]:
list_to_string(msg)

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d'

In [565]:
df2['combined'] = df2['combined'].apply(list_to_string)

In [566]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [567]:
def insert_space_at_beginning(string):
    string = ' ' + string
    return string

In [568]:
df2['overview']=df2['overview'].apply(insert_space_at_beginning)

In [889]:
df2['tags'] = df2['combined'] + df2['overview']

In [890]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [891]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [892]:
df2['original_language'] = df2['original_language'].apply(insert_space_at_beginning)

In [893]:
df2['tags'] = df2['tags'] + df2['original_language']

In [894]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [895]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization  en'

In [896]:
movies = df2[['id', 'original_title', 'tags']]

In [897]:
movies.iloc[0,2]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization  en'

In [898]:
# movies['original_title'] = movies['original_title'].apply(remove_spaces_str)

In [899]:
# movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [900]:
# movies['tags'] = movies['tags'] + movies['original_title']

In [901]:
movies.iloc[2, 2]

'DanielCraig ChristophWaltz LéaSeydoux SamMendes Action Adventure Crime spy based on novel secret agent sequel mi6 british secret service united kingdom cryptic message bond ’s past send trail uncover sinister organization battle political force keep secret service alive bond feel back layer deceit reveal terrible truth behind spectre  en'

In [902]:
msg = movies.iloc[1, 2]

In [903]:
def remove_spaces_str(string):
    string = string.replace(' ', '')
    return string

In [904]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem  en"

In [905]:
movies.head(1)

,id,original_title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [906]:
movies.iloc[1,2]

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem  en"

In [907]:
# movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [908]:
#Importing the DNN Model

In [909]:
!pip install gensim

In [910]:
import gensim

In [911]:
from gensim.utils import simple_preprocess

In [912]:
msg = movies.iloc[1, 2]

In [913]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem  en"

In [914]:
msg = remove_punc(msg)

In [915]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [916]:
doc1 = nlp(msg)

In [917]:
doc1

JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of ones life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem  en

In [918]:
#Removing punctuations from movie's titles

In [919]:
# movies['original_title'] = movies['original_title'].apply(remove_punc)

In [920]:
# movies['tags'] = movies['tags'] + movies['original_title']

In [921]:
msg = movies.iloc[1, 2]

In [922]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem  en"

In [923]:
doc1 = sent_tokenize(msg)

In [924]:
ls = []
for word in doc1:
    ls.append(simple_preprocess(word))


In [925]:
ls

[['johnnydepp',
  'orlandobloom',
  'keiraknightley',
  'goreverbinski',
  'adventure',
  'fantasy',
  'action',
  'ocean',
  'drug',
  'abuse',
  'exotic',
  'island',
  'east',
  'india',
  'trading',
  'company',
  'love',
  'of',
  'one',
  'life',
  'traitor',
  'shipwreck',
  'strong',
  'woman',
  'ship',
  'alliance',
  'calypso',
  'afterlife',
  'fighter',
  'pirate',
  'swashbuckler',
  'captain',
  'barbossa',
  'long',
  'believe',
  'dead',
  'come',
  'back',
  'life',
  'head',
  'edge',
  'earth',
  'turner',
  'elizabeth',
  'swan',
  'nothing',
  'quite',
  'seem',
  'en']]

In [926]:

def making_corpus(string):
    corpus = []
    doc1 = sent_tokenize(string)
    for word in doc1:
        corpus.append(simple_preprocess(word))

    return corpus

In [927]:
msg = movies.iloc[2, 2]

In [928]:
making_corpus(msg)

[['danielcraig',
  'christophwaltz',
  'léaseydoux',
  'sammendes',
  'action',
  'adventure',
  'crime',
  'spy',
  'based',
  'on',
  'novel',
  'secret',
  'agent',
  'sequel',
  'mi',
  'british',
  'secret',
  'service',
  'united',
  'kingdom',
  'cryptic',
  'message',
  'bond',
  'past',
  'send',
  'trail',
  'uncover',
  'sinister',
  'organization',
  'battle',
  'political',
  'force',
  'keep',
  'secret',
  'service',
  'alive',
  'bond',
  'feel',
  'back',
  'layer',
  'deceit',
  'reveal',
  'terrible',
  'truth',
  'behind',
  'spectre',
  'en']]

In [929]:
movies['tags'] = movies['tags'].apply(making_corpus)

In [930]:
movies['tags'].iloc[0]

[['samworthington',
  'zoesaldana',
  'sigourneyweaver',
  'jamescameron',
  'action',
  'adventure',
  'fantasy',
  'science',
  'fiction',
  'culture',
  'clash',
  'future',
  'space',
  'war',
  'space',
  'colony',
  'society',
  'space',
  'travel',
  'futuristic',
  'romance',
  'space',
  'alien',
  'tribe',
  'alien',
  'planet',
  'cgi',
  'marine',
  'soldier',
  'battle',
  'love',
  'affair',
  'anti',
  'war',
  'power',
  'relations',
  'mind',
  'and',
  'soul',
  'nd',
  'century',
  'paraplegic',
  'marine',
  'dispatch',
  'moon',
  'pandora',
  'unique',
  'mission',
  'become',
  'torn',
  'follow',
  'order',
  'protect',
  'alien',
  'civilization',
  'en']]

In [931]:
corpus = []

In [932]:
def corpus_formation(ls):
    corpus.append(ls)

In [933]:
movies['tags'] = movies['tags'].apply(corpus_formation)

In [934]:
from itertools import chain
flatten_corpus = list(chain.from_iterable(corpus))

In [946]:
for i in range(len(flatten_corpus)):
    if len(flatten_corpus[i]) == 0:
        print(i)

In [947]:
len(flatten_corpus)

4880

In [948]:
#Embeddings

In [949]:
#Google News Word2Vec
import gensim.downloader as api
# model2 = api.load('word2vec-google-news-300')

In [950]:
model = gensim.models.Word2Vec(window=10, min_count=1, workers=8, vector_size=300, alpha=0.03, min_alpha=0.0007)

In [951]:
model.build_vocab(flatten_corpus)

In [952]:
model.train(flatten_corpus, total_examples=model.corpus_count, epochs=10)

(2229634, 2332460)

In [953]:
# model.wv.most_similar('Spectre')

In [954]:
embeddings = model.wv.get_normed_vectors()

In [955]:
embeddings

array([[-0.00667061,  0.0554638 , -0.00925942, ..., -0.01089557,
         0.05142752,  0.00908388],
       [-0.02917327,  0.03666141, -0.08721096, ..., -0.07248939,
         0.03379479, -0.02806423],
       [ 0.00638518, -0.03161474, -0.07221184, ..., -0.0155215 ,
         0.07652514,  0.06711425],
       ...,
       [-0.0190047 ,  0.07244407, -0.09565761, ..., -0.01421657,
         0.07780842,  0.01144867],
       [-0.02884728,  0.07439642, -0.1100005 , ..., -0.01249476,
         0.0796173 ,  0.01615428],
       [-0.05667001,  0.03388194, -0.08324696, ..., -0.00626704,
         0.04056718, -0.00141739]], dtype=float32)

In [956]:
# model.wv.index_to_key

In [957]:
#Average Word2Vec for each tags

In [958]:
avg_embeddings = []

In [959]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [960]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization  en'

In [961]:
flatten_corpus[0][0]

'samworthington'

In [962]:
#Building vocabulary
vocabulary = set(model.wv.index_to_key)

In [964]:
ls = []
unique_words = []
avg_embeddings = None
vectors = []
final_embeddings = []
for i in range(len(flatten_corpus)):   
    ls = []
    unique_words = []
    vectors = []
    for j in range(len(flatten_corpus[i])):
        
        if flatten_corpus[i][j] in vocabulary: 
            
            if avg_embeddings is None:
                avg_embeddings = model.wv[flatten_corpus[i][j]]
            else:
                avg_embeddings = avg_embeddings + model.wv[flatten_corpus[i][j]]
    # ls = list(chain.from_iterable(ls))
    # unique_words = set(ls)
    # print(len(ls), len(flatten_corpus[i]))
    final_embeddings.append(avg_embeddings / len(avg_embeddings))
# print(len(list(unique_words)))

In [965]:
ls = []
ls.append(model.wv[flatten_corpus[0][0]].tolist())
# from itertools import 
ls = list(chain.from_iterable(ls))

In [966]:
len(ls)

300

In [967]:
len(flatten_corpus[0])

56

In [968]:
movies.shape

(4800, 3)

In [970]:
len(final_embeddings)

4880

In [971]:
final_embeddings[0]

array([-5.14829531e-02,  1.14777796e-01, -9.35303420e-02, -5.27457483e-02,
        3.51996571e-02,  3.86656635e-02,  1.22189939e-01,  4.43594493e-02,
        8.26359689e-02,  1.72754545e-02,  1.78036168e-02, -1.40037730e-01,
        1.46883437e-02, -5.91281690e-02,  4.70935088e-03, -4.61956263e-02,
        3.45332138e-02,  5.80490567e-02,  1.35118328e-02,  1.37831768e-04,
       -3.74701396e-02,  4.68076989e-02, -2.99559943e-02, -2.81759966e-02,
        4.29743342e-02,  4.34411764e-02, -7.68685564e-02,  1.60103862e-03,
        1.87042058e-02, -3.71420458e-02, -9.91114452e-02, -4.12325747e-02,
       -2.49973480e-02,  1.60220787e-02, -1.02085937e-02, -2.43153907e-02,
        4.19333875e-02, -4.55206037e-02,  6.88868985e-02, -5.33580855e-02,
        4.26935889e-02,  7.11045265e-02, -4.56308909e-02, -3.29872482e-02,
       -6.96806982e-03,  6.16647005e-02, -1.06159635e-01,  1.23545483e-01,
        1.07962810e-01,  3.69170755e-02, -3.93439047e-02, -1.80117600e-02,
        6.33962229e-02,  

In [972]:
import math

In [973]:
def idx_of_movie_in_embeddings(string):
    idx = movies[movies['original_title'] == string].index.values
    idx = idx.tolist()[0]
    return idx

In [974]:
idx = idx_of_movie_in_embeddings('Spider-Man')

In [975]:
idx

159

In [977]:
final_embeddings[idx]

array([ -1.5271878 ,  11.038629  , -10.671763  ,  -4.9398584 ,
         0.65617853,  -1.6575117 ,  16.073753  ,  10.064952  ,
         3.7135108 ,   2.0512707 ,   1.3824459 , -18.787565  ,
        -0.67115754,  -6.8972144 ,  -0.6164255 ,  -3.5547342 ,
         4.8426285 ,   4.210514  ,   0.801046  ,  -4.7864294 ,
        -8.473245  ,   4.403992  ,   4.187604  ,  -0.50785047,
         5.1618476 ,   5.347027  ,  -5.7591352 ,   0.06970948,
        -1.2199731 ,  -6.536593  ,  -4.9908195 ,  -2.835049  ,
         0.2878127 ,  -0.60642743,  -1.1666635 ,   1.1596926 ,
         5.836886  ,  -9.794869  ,   7.0130014 ,  -7.0143385 ,
         3.4111364 ,   7.111341  ,  -1.6777534 ,  -5.923752  ,
         0.06742737,   5.8470173 ,  -8.922242  ,  18.026018  ,
         9.382859  ,   1.8570839 ,   2.8529596 ,  -0.85638654,
         2.3306234 ,   3.9295325 ,  -5.6261864 ,  17.04962   ,
        11.103247  ,   6.236053  ,  11.973421  ,   9.603856  ,
        -5.9850736 ,  -0.05754184,  -4.209574  ,   2.46

In [978]:
#Cosine Similarity

In [979]:
from sklearn.metrics.pairwise import cosine_similarity

In [983]:
similarity_df = pd.DataFrame(final_embeddings)

In [984]:
similarity_df.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.051483,0.114778,-0.09353,-0.052746,0.0352,0.038666,0.12219,0.044359,0.082636,0.017275,0.017804,-0.140038,0.014688,-0.059128,0.004709,-0.046196,0.034533,0.058049,0.013512,0.000138,-0.03747,0.046808,-0.029956,-0.028176,0.042974,0.043441,-0.076869,0.001601,0.018704,-0.037142,-0.099111,-0.041233,-0.024997,0.016022,-0.010209,-0.024315,0.041933,-0.045521,0.068887,-0.053358,0.042694,0.071105,-0.045631,-0.032987,-0.006968,0.061665,-0.10616,0.123545,0.107963,0.036917,-0.039344,-0.018012,0.063396,0.065135,-0.068628,0.151907,0.114272,0.0395,0.078693,0.087499,-0.075413,-0.000715,-0.03473,0.004536,-0.015504,0.006749,-0.046564,0.004156,-0.013515,-0.005581,-0.153183,0.149185,0.088761,-0.046019,0.015361,-0.01865,0.012771,0.019283,-0.007943,0.022524,0.010793,0.0369,0.05795,0.140651,0.018899,0.023704,-0.072782,-0.058822,0.079521,0.034055,0.07756,0.029182,-0.018183,-0.010725,0.017642,0.126024,0.079897,-0.02721,-0.00605,-0.003879,0.017541,-0.021449,0.037232,-0.081308,0.144571,0.02688,0.075959,0.005049,0.041721,-0.016219,-0.003636,-0.014878,0.082411,0.074989,-0.039743,0.021681,-0.064908,-0.003689,0.040041,-0.059098,0.091562,-0.046195,-0.002759,0.069346,-0.028195,-0.137831,0.066456,-0.069265,0.05347,0.000775,0.055481,0.099367,-0.015293,-0.015505,0.064856,-0.000136,0.058133,-0.058598,-0.050763,-0.083365,-0.006482,0.000824,0.006497,0.084881,0.024761,0.034564,-0.095831,0.066789,0.027071,0.049672,0.063083,-0.010795,-0.002965,0.063755,-0.047064,-0.019289,-0.006098,-0.018693,-0.004063,0.119831,-0.033124,0.088733,-0.046891,0.105653,-0.08629,0.008642,-0.036939,-0.067885,-0.060306,0.075716,-0.008483,0.038323,0.042259,0.008828,-0.055875,-0.05424,0.006718,-0.171515,-0.023024,-0.042683,-0.002261,-0.05251,-0.017804,-0.01716,0.03498,0.044092,0.101087,0.061621,0.008151,0.033468,-0.040821,0.067503,-0.070813,0.018671,-0.022082,0.013762,-0.103162,-0.080081,-0.014796,0.059157,-0.01298,0.042589,0.002486,-0.086874,0.088864,0.039088,-0.029233,0.01661,0.023915,0.010275,-0.064325,-0.134842,-0.032225,0.045172,0.010686,-0.122406,0.014357,-0.007932,-0.088019,-0.07052,0.053652,0.025693,-0.149676,-0.021953,0.078794,0.009045,-0.021721,-0.063027,-0.05349,-0.050763,-0.023967,0.079904,-0.120927,-0.001116,-0.012118,0.034066,-0.07237,0.0079,0.133284,-0.070478,0.0317,-0.043633,-0.055887,0.001766,-0.018305,-0.079843,-0.048606,0.043387,0.023561,0.047152,-0.026095,-0.049561,-0.041168,-0.022924,0.002311,0.052224,-0.043679,0.122896,-0.030623,0.031622,0.01594,0.022707,-0.01771,-0.046386,-0.085493,-0.00598,-0.031566,-0.043694,0.076775,-0.025271,-0.048267,0.073543,-0.097227,-0.043387,0.114546,0.028543,0.083777,0.042709,0.014584,-0.033378,0.039609,-0.031495,-0.089116,0.000952,-0.069057,0.012308,0.007135,0.02839,0.074823,-0.025236,-0.037043,0.088949,0.017059,-0.027152,0.054542,0.010415,0.034819,-0.000887,0.031185,0.011733


In [985]:
movies.shape

(4800, 3)

In [988]:
cosine_similarity(final_embeddings[idx].reshape(1, -1), final_embeddings)

array([[0.87677115, 0.9568796 , 0.9642533 , ..., 0.9521524 , 0.9521436 ,
        0.95213073]], dtype=float32)